In [80]:
import sklearn as sk
import matplotlib as plt
import pandas as pd


# Use read_csv() to load the CSV file into a pandas DataFrame
data = pd.read_csv('Data/full_data.csv',low_memory=False)


In [81]:
columns_to_remove = ['Unnamed: 0','lastTradeDate','address1', 'city', 'country', 'industry', 'sector','recommendationKey','expirationDate','exchange','timeZoneShortName']

data.drop(columns_to_remove, axis = 1, inplace= True)


In [82]:
data = pd.get_dummies(data, columns=['ticker','state'])
data

,strike,price,bid,ask,change,percentChange,volume,openInterest,sigma,inTheMoney,...,state_UT,state_UY,state_VA,state_VIC,state_VN,state_VT,state_WA,state_WI,state_WV,state_ZA
0,210.0,199.70,417.49,418.06,0.000000,0.000000,19275689.0,0.0,0.223118,1.0,...,False,False,False,False,False,False,True,False,False,False
1,220.0,200.32,417.49,418.06,-9.569992,-4.559527,19275689.0,2.0,0.223118,1.0,...,False,False,False,False,False,False,True,False,False,False
2,230.0,189.90,417.49,418.06,1.399994,0.742702,19275689.0,3.0,0.223118,1.0,...,False,False,False,False,False,False,True,False,False,False
3,235.0,174.91,417.49,418.06,0.000000,0.000000,19275689.0,6.0,0.223118,1.0,...,False,False,False,False,False,False,True,False,False,False
4,250.0,170.23,417.49,418.06,-9.120010,-5.085035,19275689.0,6.0,0.223118,1.0,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73149,2.0,0.05,1.71,2.34,0.000000,0.000000,124821.0,1382.0,1.064334,True,...,False,False,False,False,False,False,False,False,False,False
73150,3.0,0.15,1.71,2.34,0.000000,0.000000,124821.0,12.0,1.064334,False,...,False,False,False,False,False,False,False,False,False,False
73151,4.0,0.16,1.71,2.34,0.000000,0.000000,124821.0,1.0,1.064334,False,...,False,False,False,False,False,False,False,False,False,False
73152,1.0,0.25,1.71,2.34,0.000000,0.000000,124821.0,1.0,1.064334,False,...,False,False,False,False,False,False,False,False,False,False


In [83]:
data = data.replace({"False": 0, "True": 1})
data = data.replace({False: 0, True: 1})
data = data.replace({"2024-03-08": 0})

In [84]:
ann3_data = data.replace({"2024-03-08": 0})

In [114]:
ann3_call_data = ann3_data[ann3_data.call == 1]
ann3_put_data = ann3_data[ann3_data.call == 0]

ann3_call_data.drop('call', axis = 1, inplace = True)
ann3_put_data.drop('call', axis = 1, inplace = True)

C:\Users\domen\AppData\Local\Temp\ipykernel_11888\4169316279.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ann3_call_data.drop('call', axis = 1, inplace = True)
C:\Users\domen\AppData\Local\Temp\ipykernel_11888\4169316279.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ann3_put_data.drop('call', axis = 1, inplace = True)


In [115]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam

# Assuming ann3_call_data is a pandas DataFrame
ann3_call_data = pd.DataFrame(ann3_call_data)

# Separate features and target
y = ann3_call_data['price']
X = ann3_call_data.drop("price", axis=1)

# Convert to NumPy arrays
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

# Check for NaNs and Infs before proceeding
print("Checking for NaN or Inf in X and y...")
print("NaNs in X:", np.any(np.isnan(X)))
print("Infs in X:", np.any(np.isinf(X)))
print("NaNs in y:", np.any(np.isnan(y)))
print("Infs in y:", np.any(np.isinf(y)))

# Handle NaNs and Infs
X = np.nan_to_num(X, nan=0.0, posinf=1e10, neginf=-1e10)
y = np.nan_to_num(y, nan=0.0, posinf=1e10, neginf=-1e10)

# Check again after replacement
print("NaNs in X after replacement:", np.any(np.isnan(X)))
print("Infs in X after replacement:", np.any(np.isinf(X)))
print("NaNs in y after replacement:", np.any(np.isnan(y)))
print("Infs in y after replacement:", np.any(np.isinf(y)))

# Ensure correct data type and clip large values
X = np.clip(X, -1e10, 1e10).astype(np.float32)
y = np.clip(y, -1e10, 1e10).astype(np.float32)

# Feature scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# Define the model
ANN3_call = Sequential()
ANN3_call.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
ANN3_call.add(Dense(64, activation='relu'))
ANN3_call.add(Dense(64, activation='relu'))
ANN3_call.add(Dense(1))

# Compile the model with a smaller learning rate and gradient clipping
optimizer = Adam(learning_rate=1e-4, clipnorm=1.0)
ANN3_call.compile(loss='mean_squared_error', optimizer=optimizer)

# Train the model
history = ANN3_call.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Check for NaN in loss values
loss_values = history.history['loss']
if np.any(np.isnan(loss_values)):
    print("Warning: NaN values detected in loss during training.")
else:
    print("Training completed successfully.")





Checking for NaN or Inf in X and y...
NaNs in X: False
Infs in X: True
NaNs in y: False
Infs in y: False
NaNs in X after replacement: False
Infs in X after replacement: False
NaNs in y after replacement: False
Infs in y after replacement: False
Epoch 1/50
973/973 [==============================] - 7s 6ms/step - loss: 5441.6396 - val_loss: 4893.7217
Epoch 2/50
973/973 [==============================] - 6s 6ms/step - loss: 4340.5425 - val_loss: 4251.2456
Epoch 3/50
973/973 [==============================] - 6s 6ms/step - loss: 3682.5845 - val_loss: 3308.0449
Epoch 4/50
973/973 [==============================] - 6s 7ms/step - loss: 2871.3040 - val_loss: 2771.5298
Epoch 5/50
973/973 [==============================] - 6s 6ms/step - loss: 2417.4192 - val_loss: 2250.9495
Epoch 6/50
973/973 [==============================] - 6s 6ms/step - loss: 2060.4900 - val_loss: 1925.8458
Epoch 7/50
973/973 [==============================] - 6s 6ms/step - loss: 1838.2548 - val_loss: 1744.1492
Epoch 8/50
97

In [116]:
ANN3_call.save('models\\ann3\ANN3_call.keras')

In [118]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam

# Assuming ann3_put_data is a pandas DataFrame
ann3_put_data = pd.DataFrame(ann3_put_data)

# Separate features and target
y = ann3_put_data['price']
X = ann3_put_data.drop("price", axis=1)

# Convert to NumPy arrays
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

# Check for NaNs and Infs before proceeding
print("Checking for NaN or Inf in X and y...")
print("NaNs in X:", np.any(np.isnan(X)))
print("Infs in X:", np.any(np.isinf(X)))
print("NaNs in y:", np.any(np.isnan(y)))
print("Infs in y:", np.any(np.isinf(y)))

# Handle NaNs and Infs
X = np.nan_to_num(X, nan=0.0, posinf=1e10, neginf=-1e10)
y = np.nan_to_num(y, nan=0.0, posinf=1e10, neginf=-1e10)

# Check again after replacement
print("NaNs in X after replacement:", np.any(np.isnan(X)))
print("Infs in X after replacement:", np.any(np.isinf(X)))
print("NaNs in y after replacement:", np.any(np.isnan(y)))
print("Infs in y after replacement:", np.any(np.isinf(y)))

# Ensure correct data type and clip large values
X = np.clip(X, -1e10, 1e10).astype(np.float32)
y = np.clip(y, -1e10, 1e10).astype(np.float32)

# Feature scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# Define the model
ANN3_put = Sequential()
ANN3_put.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
ANN3_put.add(Dense(64, activation='relu'))
ANN3_put.add(Dense(64, activation='relu'))
ANN3_put.add(Dense(1))

# Compile the model with a smaller learning rate and gradient clipping
optimizer = Adam(learning_rate=1e-4, clipnorm=1.0)
ANN3_put.compile(loss='mean_squared_error', optimizer=optimizer)

# Train the model
history = ANN3_put.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Check for NaN in loss values
loss_values = history.history['loss']
if np.any(np.isnan(loss_values)):
    print("Warning: NaN values detected in loss during training.")
else:
    print("Training completed successfully.")





Checking for NaN or Inf in X and y...
NaNs in X: False
Infs in X: True
NaNs in y: False
Infs in y: False
NaNs in X after replacement: False
Infs in X after replacement: False
NaNs in y after replacement: False
Infs in y after replacement: False
Epoch 1/50
857/857 [==============================] - 8s 7ms/step - loss: 1632.0319 - val_loss: 1280.8892
Epoch 2/50
857/857 [==============================] - 7s 8ms/step - loss: 1533.2502 - val_loss: 1154.5188
Epoch 3/50
857/857 [==============================] - 7s 8ms/step - loss: 1412.9767 - val_loss: 1032.3654
Epoch 4/50
857/857 [==============================] - 7s 8ms/step - loss: 1279.5116 - val_loss: 899.8011
Epoch 5/50
857/857 [==============================] - 7s 8ms/step - loss: 1149.7511 - val_loss: 808.1373
Epoch 6/50
857/857 [==============================] - 7s 8ms/step - loss: 1048.6151 - val_loss: 760.3099
Epoch 7/50
857/857 [==============================] - 7s 8ms/step - loss: 969.3928 - val_loss: 682.9295
Epoch 8/50
857/857

In [119]:
ANN3_put.save('models\\ann3\ANN3_put.keras')

#### PERFORMANCE FOR CALL MODEL

In [120]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam

# Assuming ann3_call_data is a pandas DataFrame
ann3_call_data = pd.DataFrame(ann3_call_data)

# Separate features and target
y = ann3_call_data['price']
X = ann3_call_data.drop("price", axis=1)

# Convert to NumPy arrays
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

# Check for NaNs and Infs before proceeding
print("Checking for NaN or Inf in X and y...")
print("NaNs in X:", np.any(np.isnan(X)))
print("Infs in X:", np.any(np.isinf(X)))
print("NaNs in y:", np.any(np.isnan(y)))
print("Infs in y:", np.any(np.isinf(y)))

# Handle NaNs and Infs
X = np.nan_to_num(X, nan=0.0, posinf=1e10, neginf=-1e10)
y = np.nan_to_num(y, nan=0.0, posinf=1e10, neginf=-1e10)

# Check again after replacement
print("NaNs in X after replacement:", np.any(np.isnan(X)))
print("Infs in X after replacement:", np.any(np.isinf(X)))
print("NaNs in y after replacement:", np.any(np.isnan(y)))
print("Infs in y after replacement:", np.any(np.isinf(y)))

# Ensure correct data type and clip large values
X = np.clip(X, -1e10, 1e10).astype(np.float32)
y = np.clip(y, -1e10, 1e10).astype(np.float32)

# Feature scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

Checking for NaN or Inf in X and y...
NaNs in X: False
Infs in X: True
NaNs in y: False
Infs in y: False
NaNs in X after replacement: False
Infs in X after replacement: False
NaNs in y after replacement: False
Infs in y after replacement: False


In [121]:
y_pred = ANN3_call.predict(X_test)

244/244 [==============================] - 1s 3ms/step


In [122]:
y_pred.shape

(7779, 1)

In [123]:
y_test.shape

(7779,)

In [124]:
df = pd.DataFrame()
df['price'] = y_test
df['ANN3'] = y_pred

In [125]:
df

,price,ANN3
0,14.500000,41.336864
1,0.010000,-0.232550
2,5.300000,2.930640
3,0.030000,0.497290
4,0.100000,6.392140
...,...,...
7774,0.040000,-1.884610
7775,27.299999,18.069721
7776,1.750000,2.912940
7777,0.190000,0.060794


In [126]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

# Assuming 'df' is your DataFrame and 'ANN3' represents your predictions
# Calculate Mean Squared Error (MSE)
ANN3_call_mse = mean_squared_error(df['price'], df['ANN3'])

# Calculate Root Mean Squared Error (RMSE)
ANN3_call_rmse = np.sqrt(ANN3_call_mse)

# Calculate Mean Absolute Error (MAE)
ANN3_call_mae = mean_absolute_error(df['price'], df['ANN3'])

# Calculate Mean Absolute Percentage Error (MAPE)
ANN3_call_mape = np.mean(np.abs((df['price'] - df['ANN3']) / df['price'])) * 100

# Calculate R-squared
ANN3_call_r_squared = r2_score(df['price'], df['ANN3'])

print("Mean Squared Error (MSE):", ANN3_call_mse)
print("Root Mean Squared Error (RMSE):", ANN3_call_rmse)
print("Mean Absolute Error (MAE):", ANN3_call_mae)
print("Mean Absolute Percentage Error (MAPE):", ANN3_call_mape)
print("R-squared:", ANN3_call_r_squared)



Mean Squared Error (MSE): 358.99982
Root Mean Squared Error (RMSE): 18.94729
Mean Absolute Error (MAE): 5.715891
Mean Absolute Percentage Error (MAPE): 839.8326873779297
R-squared: 0.9450504856321826


#### PERFORMANCE FOR PUT MODEL

In [127]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam

# Assuming ann3_call_data is a pandas DataFrame
ann3_put_data = pd.DataFrame(ann3_put_data)

# Separate features and target
y = ann3_put_data['price']
X = ann3_put_data.drop("price", axis=1)

# Convert to NumPy arrays
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

# Check for NaNs and Infs before proceeding
print("Checking for NaN or Inf in X and y...")
print("NaNs in X:", np.any(np.isnan(X)))
print("Infs in X:", np.any(np.isinf(X)))
print("NaNs in y:", np.any(np.isnan(y)))
print("Infs in y:", np.any(np.isinf(y)))

# Handle NaNs and Infs
X = np.nan_to_num(X, nan=0.0, posinf=1e10, neginf=-1e10)
y = np.nan_to_num(y, nan=0.0, posinf=1e10, neginf=-1e10)

# Check again after replacement
print("NaNs in X after replacement:", np.any(np.isnan(X)))
print("Infs in X after replacement:", np.any(np.isinf(X)))
print("NaNs in y after replacement:", np.any(np.isnan(y)))
print("Infs in y after replacement:", np.any(np.isinf(y)))

# Ensure correct data type and clip large values
X = np.clip(X, -1e10, 1e10).astype(np.float32)
y = np.clip(y, -1e10, 1e10).astype(np.float32)

# Feature scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

Checking for NaN or Inf in X and y...
NaNs in X: False
Infs in X: True
NaNs in y: False
Infs in y: False
NaNs in X after replacement: False
Infs in X after replacement: False
NaNs in y after replacement: False
Infs in y after replacement: False


In [128]:
y_pred = ANN3_put.predict(X_test)

215/215 [==============================] - 1s 2ms/step


In [129]:
df = pd.DataFrame()
df['price'] = y_test
df['ANN3'] = y_pred

In [130]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

# Assuming 'df' is your DataFrame and 'ANN3' represents your predictions
# Calculate Mean Squared Error (MSE)
ANN3_put_mse = mean_squared_error(df['price'], df['ANN3'])

# Calculate Root Mean Squared Error (RMSE)
ANN3_put_rmse = np.sqrt(ANN3_put_mse)

# Calculate Mean Absolute Error (MAE)
ANN3_put_mae = mean_absolute_error(df['price'], df['ANN3'])

# Calculate Mean Absolute Percentage Error (MAPE)
ANN3_put_mape = np.mean(np.abs((df['price'] - df['ANN3']) / df['price'])) * 100

# Calculate R-squared
ANN3_put_r_squared = r2_score(df['price'], df['ANN3'])

print("Mean Squared Error (MSE):", ANN3_put_mse)
print("Root Mean Squared Error (RMSE):", ANN3_put_rmse)
print("Mean Absolute Error (MAE):", ANN3_put_mae)
print("Mean Absolute Percentage Error (MAPE):", ANN3_put_mape)
print("R-squared:", ANN3_put_r_squared)


Mean Squared Error (MSE): 460.22668
Root Mean Squared Error (RMSE): 21.452894
Mean Absolute Error (MAE): 5.165891
Mean Absolute Percentage Error (MAPE): 1410.607624053955
R-squared: 0.6648965254284634
